Importing Pandas and loading our comma seperated value data sets.

In [640]:
import pandas as pd

In [641]:
df1= pd.read_csv('users.csv', encoding='latin1')
df1.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [642]:
df2= pd.read_csv('engagement.csv')
df2.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [643]:
df2.drop(['visited'], axis=1, inplace=True)
df2['time_stamp']=pd.to_datetime(df2['time_stamp'])
df2.head()

,time_stamp,user_id
0,2014-04-22 03:53:30,1
1,2013-11-15 03:45:04,2
2,2013-11-29 03:45:04,2
3,2013-12-09 03:45:04,2
4,2013-12-25 03:45:04,2


Need to create a column that contains a feature for adopted users.

In [644]:
from datetime import timedelta, datetime

def user(x):
    if len (x) >=3:
        x=[i for i in x] # creating a list for x
        x.sort() # placing dates in increasing order
        x= [x[i+1] - x[1] for i in range(len(x)-2)] 
        x= [i for i in range(len(x)-2) if x[i+1] + x[1] + x[i+2] <= timedelta(days=7)]
        if 1 in x:
            return 1

In [645]:
engage= df2.groupby('user_id').agg(user)
engage.fillna(0, inplace=True)
engage.columns = ['adopted_user']
engage.reset_index()
engage.head()

,adopted_user
user_id,
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0


Need to join our newly created column adopted users to our existing pandas dataframe.

In [649]:
matrix= df1.join(engage, how='left')
matrix.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,NaN
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,0.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,0.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,0.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,0.0


Filled in our null values with zero and checked to make sure no more null values exist.

In [650]:
matrix.fillna(0, inplace=True)
matrix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 11 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    12000 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            12000 non-null float64
adopted_user                  12000 non-null float64
dtypes: float64(3), int64(4), object(4)
memory usage: 1.0+ MB


Turning each source into a dummy variable in order to use this information in our machine learning model.

In [651]:
dummy=pd.get_dummies(matrix['creation_source'])
m1= pd.concat([matrix, dummy], axis=1)

In [652]:
m2=m1.drop(['name', 'email', 'creation_time', 'last_session_creation_time','creation_source'], axis=1)
m2.head()

,object_id,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user,GUEST_INVITE,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH
0,1,1,0,11,10803.0,0.0,1,0,0,0,0
1,2,0,0,1,316.0,0.0,0,1,0,0,0
2,3,0,0,94,1525.0,0.0,0,1,0,0,0
3,4,0,0,1,5151.0,0.0,1,0,0,0,0
4,5,0,0,193,5240.0,0.0,1,0,0,0,0


In [653]:
m2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 11 columns):
object_id                     12000 non-null int64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            12000 non-null float64
adopted_user                  12000 non-null float64
GUEST_INVITE                  12000 non-null uint8
ORG_INVITE                    12000 non-null uint8
PERSONAL_PROJECTS             12000 non-null uint8
SIGNUP                        12000 non-null uint8
SIGNUP_GOOGLE_AUTH            12000 non-null uint8
dtypes: float64(2), int64(4), uint8(5)
memory usage: 621.2 KB


In [654]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

Standardized our data using MinMaxScaler and then trained the data in order to test and fit it to our model.

In [656]:
X= m2.drop(['object_id','org_id','adopted_user'], axis=1)
y= m2['adopted_user']

scale=StandardScaler()
scale.fit(x_train, y_train)
scaler.fit_transform(X)

x_train, X_test, y_train, y_test= train_test_split(X, y, train_size=0.30, random_state=42)

tree=DecisionTreeClassifier()
tree.fit(x_train, y_train)

y_pred= tree.predict(X_test)

for idx, val in enumerate(tree.feature_importances_):
    print('{:30}{:3f}'.format(X.columns[idx], val))
    
print('{:30}{:3f}'.format('f1 score', f1_score(y_test, y_pred)))
print('{:30}{:3f}'.format('accuracy score', accuracy_score(y_test, y_pred)))
print()


opted_in_to_mailing_list      0.168137
enabled_for_marketing_drip    0.144786
invited_by_user_id            0.638684
GUEST_INVITE                  0.000000
ORG_INVITE                    0.047884
PERSONAL_PROJECTS             0.000000
SIGNUP                        0.000000
SIGNUP_GOOGLE_AUTH            0.000509
f1 score                      0.000000
accuracy score                0.997976



C:\Users\Kevin Bryson\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Results:

According to out DecisionTreeClassifier, it would seem that adopted users were gained by using a mailing list, guest invites or the best feature, invites by another user. I would reccomend that offer incentives for our current users that can successfully invite non-users to become adopted users.
